In [18]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import requests
import datetime
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [20]:
load_dotenv()

True

In [21]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml


In [22]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'personal', 'session']


In [23]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [ ]:
from sleeping_beauty.oura.auth.domain.auth_preflight_result import AuthPreflightReport


storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
preflight: AuthPreflightReport = oura_auth.preflight_check()
print("\n".join(preflight.messages))

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~43166 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [25]:
# ---------------------------------------------------------------------
# Date window (keep small for testing)
# ---------------------------------------------------------------------

end_date = date.today()
start_date = end_date - timedelta(days=3)

print("Testing sleep endpoint from", start_date, "to", end_date)


# ---------------------------------------------------------------------
# 1. Fetch one page
# ---------------------------------------------------------------------

page = await client.get_sleep_page(
    start_date=start_date,
    end_date=end_date,
)

print("Fetched page:")
print("  items:", len(page.data))
print("  next_token:", page.next_token)


# Basic sanity checks
assert page.data is not None
assert isinstance(page.data, list | tuple)


# ---------------------------------------------------------------------
# 2. Inspect first document (if any)
# ---------------------------------------------------------------------

if page.data:
    doc = page.data[0]

    print("\nFirst sleep document:")
    print("  id:", doc.id)
    print("  day:", doc.day)
    print("  type:", doc.type)
    print("  period:", doc.period)
    print("  time_in_bed:", doc.time_in_bed)
    print("  total_sleep_duration:", doc.total_sleep_duration)
    print("  efficiency:", doc.efficiency)
else:
    print("\nNo sleep documents returned in this range.")


# ---------------------------------------------------------------------
# 3. Iterate via async iterator (pagination test)
# ---------------------------------------------------------------------

all_docs = [
    item
    async for item in client.iter_sleep(
        start_date=start_date,
        end_date=end_date,
    )
]

print("\nIterated documents:", len(all_docs))


# ---------------------------------------------------------------------
# 4. Single-document fetch (round-trip validation)
# ---------------------------------------------------------------------

if all_docs:
    first = all_docs[0]

    single = await client.get_sleep(document_id=first.id)

    print("\nSingle-document fetch:")
    print("  id:", single.id)
    print("  matches:", single.id == first.id)

    # Hard equality checks on invariant fields
    assert single.id == first.id
    assert single.day == first.day
    assert single.period == first.period
    assert single.type == first.type
else:
    print("\nNo documents available to test single-document fetch.")

Testing sleep endpoint from 2026-01-08 to 2026-01-11
Fetched page:
  items: 6
  next_token: None

First sleep document:
  id: b6b20e6c-d8d1-4c42-b849-2038493fe2c5
  day: 2026-01-09
  type: sleep
  period: 0
  time_in_bed: 2520
  total_sleep_duration: 840
  efficiency: 33

Iterated documents: 6

Single-document fetch:
  id: b6b20e6c-d8d1-4c42-b849-2038493fe2c5
  matches: True
